<a href="https://colab.research.google.com/github/BookHaven12/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimization 1- Use Keras Tuner to determine the optimal amount of layers and neurons

## Preprocessing

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop columns 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [7]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [8]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [9]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C1248,1
C6100,1
C1820,1


In [10]:
# Look at CLASSIFICATION value counts >1
counts = application_df['CLASSIFICATION'].value_counts()
filtered_counts = counts[counts>1]
filtered_counts

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# .index filters down to the classification names to replace (creating a list of classification names that appear less than 1000 times in the data)
classifications_to_replace = counts[counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [89]:
# name_counts = application_df['NAME'].value_counts()
# filtered_name = name_counts[name_counts <= 5]
# filtered_name

,count
NAME,
AGGIELAND PETS WITH A PURPOSE,1
THE FOUNDATION FOR NORTH AMERICA WILD SHEEP,1
MIDAMERICA TEACHERS OF ENGLISH TO SPEAKERS OF OTHER LANGUAGES INC,1
AMERICAN MOCK TRIAL ASSOCIATION,1
C RICHARD & JOAN E STARK SUPPORTING FOUNDATION,1
...,...
CALVARY ACADEMY-FLORIDA INC,1
NORTH CHANNEL ROC RENEWING OUR COMMUNITY INC,1
SPOA 1849 FOUNDATION,1


In [93]:
# # Choose a cutoff value and create a list of classifications to be replaced
# # .index filters down to the classification names to replace (creating a list of classification names that appear less than or equal to 5 times in the data)
# names_to_replace = list(name_counts[name_counts <= 5].index)

# # Replace in dataframe
# for cls in names_to_replace:
#     application_df['NAME'] = application_df['NAME'].replace(cls,"Other")

# # Check to make sure replacement was successful
# application_df['NAME'].value_counts()

,count
NAME,
Other,20043
PARENT BOOSTER USA INC,1260
TOPS CLUB INC,765
UNITED STATES BOWLING CONGRESS INC,700
WASHINGTON STATE UNIVERSITY,492
...,...
SOCIETY FOR CREATIVE ANACHRONISM,6
CBMC INC,6
FAMILY CAREER AND COMMUNITY LEADERS OF AMERICA INC,6


In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
classification_dummies = pd.get_dummies(application_df)

In [13]:
# Split our preprocessed data into our features and target arrays
y = classification_dummies["IS_SUCCESSFUL"]
X = classification_dummies.drop(columns="IS_SUCCESSFUL", axis="columns").values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
classification_dummies.shape[1]

44

## Compile, Train and Evaluate the Model

In [16]:
# This counts how many features (columns) your data has, so the model knows what kind of input to expect.
number_input_features = len(X_train[0])

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','selu'])

    # Allow kerastuner to decide number of neurons in first layer
    nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=8), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 4)):
        nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=4,
            max_value=64,
            step=8),
            activation=activation))

    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn

In [17]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [18]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 10s]
val_accuracy: 0.7307288646697998

Best val_accuracy So Far: 0.7335277199745178
Total elapsed time: 00h 29m 45s


In [20]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 65,
 'num_layers': 4,
 'units_0': 60,
 'units_1': 28,
 'units_2': 44,
 'units_3': 36,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [26]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 5ms/step - accuracy: 0.7335 - loss: 0.5535
Loss: 0.5535439848899841, Accuracy: 0.7335277199745178


In [27]:
# Save the model to a file
best_model.save('AlphabetSoupCharity_Optimization_test_1.keras')

### Model 1 Results:
**73 % Accuracy and 55 % Loss**

# Optimization Test #2 - ADD 'NAME' column and use the optimized layers and neurons from the previous test


### Preprocessing for Model 2

In [28]:
# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# Drop 'EIN' column only, ADD back in 'NAME'
application_df = application_df.drop(columns = ['EIN'])

# Create a list of application types to be replaced
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

# Look at CLASSIFICATION value counts >1
counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced
# .index filters down to the classification names to replace (creating a list of classification names that appear less than 1000 times in the data)
classifications_to_replace = counts[counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [29]:
# Look at NAME value counts and filter less than 5
name_counts = application_df['NAME'].value_counts()
filtered_name = name_counts[name_counts <= 5]
filtered_name

,count
NAME,
ACTS MINISTRY,5
NATIONAL ORGANIZATION FOR WOMEN INC,5
INTERNATIONL TRANSPLANT NURSES SOCIETY,5
ACTS MISSIONS,5
WARRIOR BROTHERHOOD OF VETERANS MOTORCYCLE CLUB INC,5
...,...
OUR PERFECT STORM,1
FOUNTAIN THEATER,1
COMBAT VETERANS MOTORCYCLE ASSOCIATION 33-7,1


In [30]:
# Choose a cutoff value and create a list of NAMES to be replaced
names_to_replace = list(name_counts[name_counts <= 5].index)

# Replace in dataframe
for cls in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['NAME'].value_counts()

,count
NAME,
Other,20043
PARENT BOOSTER USA INC,1260
TOPS CLUB INC,765
UNITED STATES BOWLING CONGRESS INC,700
WASHINGTON STATE UNIVERSITY,492
...,...
SOCIETY FOR CREATIVE ANACHRONISM,6
CBMC INC,6
FAMILY CAREER AND COMMUNITY LEADERS OF AMERICA INC,6


In [31]:
# Convert categorical data to numeric with `pd.get_dummies`
classification_dummies = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
y = classification_dummies["IS_SUCCESSFUL"]
X = classification_dummies.drop(columns="IS_SUCCESSFUL", axis="columns").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate Model 2

In [33]:
# Create the Model
# This counts how many features (columns) your data has, so the model knows what kind of input to expect.
number_input_features = len(X_train[0])

hidden_nodes_layer1 = 65
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 28
hidden_nodes_layer4 = 44
hidden_nodes_layer5 = 36

# Start building the Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim= number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer2, activation="relu"))

# Third Hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer3, activation="relu"))

# Fourth Hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer4, activation="relu"))

# Fifth Hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 65)             │        25,935 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 60)             │         3,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 28)             │         1,708 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 44)             │         1,276 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 36)             │         1,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            37 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,536 (134.91 KB)

 Trainable params: 34,536 (134.91 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7446 - loss: 0.5223
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7953 - loss: 0.4298
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7975 - loss: 0.4230
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7963 - loss: 0.4247
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7973 - loss: 0.4232
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7982 - loss: 0.4216
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7956 - loss: 0.4217
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8011 - loss: 0.4200
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8003 - loss: 0.4178
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8000 - loss: 0.4187
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8037 - loss: 0.4148
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#Save File
nn.save('AlphabetSoupCharity_Optimization_test_2.keras')

268/268 - 2s - 7ms/step - accuracy: 0.7914 - loss: 0.4997
Loss: 0.49969062209129333, Accuracy: 0.7913702726364136


### Model 2 Results:  
**79 % Accuracy and 49.97 % Loss**

# Optimization Test #3- Change Activiation to LeakyReLu

### Preprocessing for Model 3

In [35]:
# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# Drop 'EIN' column
application_df = application_df.drop(columns = ['EIN'])

# Create a list of application types to be replaced
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

# Look at CLASSIFICATION value counts >1
counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced
# .index filters down to the classification names to replace (creating a list of classification names that appear less than 1000 times in the data)
classifications_to_replace = counts[counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [36]:
# Filter name counts less than 5 and Create a list of NAMES to be replaced
name_counts = application_df['NAME'].value_counts()
names_to_replace = list(name_counts[name_counts <= 5].index)

# Replace in dataframe
for cls in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['NAME'].value_counts()

,count
NAME,
Other,20043
PARENT BOOSTER USA INC,1260
TOPS CLUB INC,765
UNITED STATES BOWLING CONGRESS INC,700
WASHINGTON STATE UNIVERSITY,492
...,...
SOCIETY FOR CREATIVE ANACHRONISM,6
CBMC INC,6
FAMILY CAREER AND COMMUNITY LEADERS OF AMERICA INC,6


In [37]:
# Convert categorical data to numeric with `pd.get_dummies`
classification_dummies = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
y = classification_dummies["IS_SUCCESSFUL"]
X = classification_dummies.drop(columns="IS_SUCCESSFUL", axis="columns").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate Model 3

In [38]:
from tensorflow.keras.layers import Dense, LeakyReLU

# Count number of input features
number_input_features = len(X_train[0])

# Define hidden layer sizes
hidden_nodes_layer1 = 65
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 28
hidden_nodes_layer4 = 44
hidden_nodes_layer5 = 36

# Build the model
nn = tf.keras.models.Sequential()

# Input layer + Leaky ReLU
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features))
nn.add(LeakyReLU(alpha=0.1))

# Hidden layers + Leaky ReLU
nn.add(Dense(units=hidden_nodes_layer2))
nn.add(LeakyReLU(alpha=0.1))

nn.add(Dense(units=hidden_nodes_layer3))
nn.add(LeakyReLU(alpha=0.1))

nn.add(Dense(units=hidden_nodes_layer4))
nn.add(LeakyReLU(alpha=0.1))

nn.add(Dense(units=hidden_nodes_layer5))
nn.add(LeakyReLU(alpha=0.1))

# Output layer
nn.add(Dense(units=1, activation="sigmoid"))

# Print summary
nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 65)             │        25,935 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 65)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 60)             │         3,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 60)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 28)             │         1,708 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 44)             │         1,276 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 44)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 36)             │         1,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 36)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            37 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,536 (134.91 KB)

 Trainable params: 34,536 (134.91 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
# Compile model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7433 - loss: 0.5224
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7947 - loss: 0.4275
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7998 - loss: 0.4206
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8009 - loss: 0.4194
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7968 - loss: 0.4217
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8002 - loss: 0.4179
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7945 - loss: 0.4247
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8020 - loss: 0.4178
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8000 - loss: 0.4182
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7979 - loss: 0.4191
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7968 - loss: 0.4200
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#Save File
nn.save('AlphabetSoupCharity_Optimization_test_3.keras')

268/268 - 1s - 4ms/step - accuracy: 0.7927 - loss: 0.4722
Loss: 0.4722135365009308, Accuracy: 0.7926530838012695


### Model 3 Accuracy:
**79 % Accuracy and 47% Loss**

# Final Resutls from the 3 tests and Models:

* Model 1 Keras Tuner:  **73% Accuracy and 55% Loss**
* Model 2 Adding back 'NAME':  **79% Accuracy and 49.97% Loss**
* Model 3 Leaky ReLu Activation:  **79% Accuracy and 47% Loss**